<a href="https://colab.research.google.com/github/ejlanz/Zindi_Xente_Fraud_Detection_Challenge/blob/master/Xente_Fraud_Detection_Challenge_EJ_Lanz_basic_RF_tuned.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!pip install -U -q PyDrive
 
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
 
# 1. Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [0]:
file_list = drive.ListFile({'q': "'root' in parents and trashed=false"}).GetList()
#for file1 in file_list:
#  print('title: %s, id: %s' % (file1['title'], file1['id']))

In [0]:
train_downloaded = drive.CreateFile({'id': '16TXVkIMx2zjCnzUrTLuMaVNxjTTMYaAN'})
train_downloaded.GetContentFile('training.csv')
test_downloaded = drive.CreateFile({'id': '18swBTWE6Ap8DX6V_ZTyDGvZ5tjbH8fVN'})
test_downloaded.GetContentFile('test.csv') 

In [0]:
#Importing the libraries
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import pandas_profiling
import datetime


# Model Building

    #classifiers
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.ensemble import AdaBoostClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import RandomForestClassifier

    #training features
from sklearn.model_selection import cross_val_score, train_test_split
from sklearn.model_selection import GridSearchCV

    #performance measures
from sklearn.metrics import accuracy_score,log_loss
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.metrics import make_scorer

    #filter future warnings
#two futre warnings occured multiple times when running cross validation and GridSearchCV have been removed
#FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
#FutureWarning: The default value of cv will change from 3 to 5 in version 0.22
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)


In [12]:
train = pd.read_csv('training.csv')
train.head()

,TransactionId,BatchId,AccountId,SubscriptionId,CustomerId,CurrencyCode,CountryCode,ProviderId,ProductId,ProductCategory,ChannelId,Amount,Value,TransactionStartTime,PricingStrategy,FraudResult
0,TransactionId_76871,BatchId_36123,AccountId_3957,SubscriptionId_887,CustomerId_4406,UGX,256,ProviderId_6,ProductId_10,airtime,ChannelId_3,1000.0,1000,2018-11-15T02:18:49Z,2,0
1,TransactionId_73770,BatchId_15642,AccountId_4841,SubscriptionId_3829,CustomerId_4406,UGX,256,ProviderId_4,ProductId_6,financial_services,ChannelId_2,-20.0,20,2018-11-15T02:19:08Z,2,0
2,TransactionId_26203,BatchId_53941,AccountId_4229,SubscriptionId_222,CustomerId_4683,UGX,256,ProviderId_6,ProductId_1,airtime,ChannelId_3,500.0,500,2018-11-15T02:44:21Z,2,0
3,TransactionId_380,BatchId_102363,AccountId_648,SubscriptionId_2185,CustomerId_988,UGX,256,ProviderId_1,ProductId_21,utility_bill,ChannelId_3,20000.0,21800,2018-11-15T03:32:55Z,2,0
4,TransactionId_28195,BatchId_38780,AccountId_4841,SubscriptionId_3829,CustomerId_988,UGX,256,ProviderId_4,ProductId_6,financial_services,ChannelId_2,-644.0,644,2018-11-15T03:34:21Z,2,0


In [13]:
test = pd.read_csv('test.csv')
test.head()

,TransactionId,BatchId,AccountId,SubscriptionId,CustomerId,CurrencyCode,CountryCode,ProviderId,ProductId,ProductCategory,ChannelId,Amount,Value,TransactionStartTime,PricingStrategy
0,TransactionId_50600,BatchId_35028,AccountId_2441,SubscriptionId_4426,CustomerId_2857,UGX,256,ProviderId_5,ProductId_3,airtime,ChannelId_3,1000.0,1000,2019-02-13T10:01:40Z,4
1,TransactionId_95109,BatchId_45139,AccountId_3439,SubscriptionId_2643,CustomerId_3874,UGX,256,ProviderId_5,ProductId_15,financial_services,ChannelId_3,2000.0,2000,2019-02-13T10:02:12Z,2
2,TransactionId_47357,BatchId_74887,AccountId_4841,SubscriptionId_3829,CustomerId_2857,UGX,256,ProviderId_4,ProductId_6,financial_services,ChannelId_2,-50.0,50,2019-02-13T10:02:30Z,2
3,TransactionId_28185,BatchId_11025,AccountId_2685,SubscriptionId_4626,CustomerId_3105,UGX,256,ProviderId_5,ProductId_10,airtime,ChannelId_3,3000.0,3000,2019-02-13T10:02:38Z,4
4,TransactionId_22140,BatchId_29804,AccountId_4841,SubscriptionId_3829,CustomerId_3105,UGX,256,ProviderId_4,ProductId_6,financial_services,ChannelId_2,-60.0,60,2019-02-13T10:02:58Z,2


In [14]:
sns.countplot(x=train['FraudResult'])
#plt.show()

In [15]:
train[train['FraudResult']==1].count()

TransactionId           193
BatchId                 193
AccountId               193
SubscriptionId          193
CustomerId              193
CurrencyCode            193
CountryCode             193
ProviderId              193
ProductId               193
ProductCategory         193
ChannelId               193
Amount                  193
Value                   193
TransactionStartTime    193
PricingStrategy         193
FraudResult             193
dtype: int64

In [16]:
train[train['FraudResult']==0].count()

TransactionId           95469
BatchId                 95469
AccountId               95469
SubscriptionId          95469
CustomerId              95469
CurrencyCode            95469
CountryCode             95469
ProviderId              95469
ProductId               95469
ProductCategory         95469
ChannelId               95469
Amount                  95469
Value                   95469
TransactionStartTime    95469
PricingStrategy         95469
FraudResult             95469
dtype: int64

In [0]:
#create Set column
test['Set'] = 'test'
train['Set'] = 'train'

In [0]:
# set index to TransactionID
train.set_index('TransactionId',inplace=True)
test.set_index('TransactionId',inplace=True)

In [19]:
train.head()

,BatchId,AccountId,SubscriptionId,CustomerId,CurrencyCode,CountryCode,ProviderId,ProductId,ProductCategory,ChannelId,Amount,Value,TransactionStartTime,PricingStrategy,FraudResult,Set
TransactionId,,,,,,,,,,,,,,,,
TransactionId_76871,BatchId_36123,AccountId_3957,SubscriptionId_887,CustomerId_4406,UGX,256,ProviderId_6,ProductId_10,airtime,ChannelId_3,1000.0,1000,2018-11-15T02:18:49Z,2,0,train
TransactionId_73770,BatchId_15642,AccountId_4841,SubscriptionId_3829,CustomerId_4406,UGX,256,ProviderId_4,ProductId_6,financial_services,ChannelId_2,-20.0,20,2018-11-15T02:19:08Z,2,0,train
TransactionId_26203,BatchId_53941,AccountId_4229,SubscriptionId_222,CustomerId_4683,UGX,256,ProviderId_6,ProductId_1,airtime,ChannelId_3,500.0,500,2018-11-15T02:44:21Z,2,0,train
TransactionId_380,BatchId_102363,AccountId_648,SubscriptionId_2185,CustomerId_988,UGX,256,ProviderId_1,ProductId_21,utility_bill,ChannelId_3,20000.0,21800,2018-11-15T03:32:55Z,2,0,train
TransactionId_28195,BatchId_38780,AccountId_4841,SubscriptionId_3829,CustomerId_988,UGX,256,ProviderId_4,ProductId_6,financial_services,ChannelId_2,-644.0,644,2018-11-15T03:34:21Z,2,0,train


In [20]:
test.head()

,BatchId,AccountId,SubscriptionId,CustomerId,CurrencyCode,CountryCode,ProviderId,ProductId,ProductCategory,ChannelId,Amount,Value,TransactionStartTime,PricingStrategy,Set
TransactionId,,,,,,,,,,,,,,,
TransactionId_50600,BatchId_35028,AccountId_2441,SubscriptionId_4426,CustomerId_2857,UGX,256,ProviderId_5,ProductId_3,airtime,ChannelId_3,1000.0,1000,2019-02-13T10:01:40Z,4,test
TransactionId_95109,BatchId_45139,AccountId_3439,SubscriptionId_2643,CustomerId_3874,UGX,256,ProviderId_5,ProductId_15,financial_services,ChannelId_3,2000.0,2000,2019-02-13T10:02:12Z,2,test
TransactionId_47357,BatchId_74887,AccountId_4841,SubscriptionId_3829,CustomerId_2857,UGX,256,ProviderId_4,ProductId_6,financial_services,ChannelId_2,-50.0,50,2019-02-13T10:02:30Z,2,test
TransactionId_28185,BatchId_11025,AccountId_2685,SubscriptionId_4626,CustomerId_3105,UGX,256,ProviderId_5,ProductId_10,airtime,ChannelId_3,3000.0,3000,2019-02-13T10:02:38Z,4,test
TransactionId_22140,BatchId_29804,AccountId_4841,SubscriptionId_3829,CustomerId_3105,UGX,256,ProviderId_4,ProductId_6,financial_services,ChannelId_2,-60.0,60,2019-02-13T10:02:58Z,2,test


In [0]:
set_col = ['Set']

classify_col = ['FraudResult']

numerical_list = ['Value']

dummy_list = ['ProviderId', 'ProductId','ProductCategory', 'ChannelId', 'PricingStrategy']

calc_list = ['TransactionStartTime']

delete_list = ['BatchId', 'AccountId', 'SubscriptionId', 'CustomerId', 'CurrencyCode', 'CountryCode','Amount']

In [22]:
select_list = set_col + classify_col + numerical_list + calc_list + dummy_list
select_list

['Set',
 'FraudResult',
 'Value',
 'TransactionStartTime',
 'ProviderId',
 'ProductId',
 'ProductCategory',
 'ChannelId',
 'PricingStrategy']

In [0]:
def manipulate_data(df):
  #select variables
  new_df = df[select_list]
  
  new_df['value_string'] = new_df['Value'].astype(str)
  
  new_df['value_last3'] = new_df['value_string'].str.strip().str[-3:]
  new_df['value_last4'] = new_df['value_string'].str.strip().str[-4:]
  new_df['value_last5'] = new_df['value_string'].str.strip().str[-5:]
  new_df['value_last6'] = new_df['value_string'].str.strip().str[-6:]
  
  
  # values with zero at the end
  new_df['Last_3'] = new_df['value_last3'].apply(lambda x: 'Zeroes' if x == '000' else 'Not 0s')
  new_df['Last_4'] = new_df['value_last4'].apply(lambda x: 'Zeroes' if x == '0000' else 'Not 0s')
  new_df['Last_5'] = new_df['value_last5'].apply(lambda x: 'Zeroes' if x == '00000' else 'Not 0s')
  new_df['Last_6'] = new_df['value_last6'].apply(lambda x: 'Zeroes' if x == '000000' else 'Not 0s')
  
  #date time into hour of the day
  new_df['TransactionStartTime'] = pd.to_datetime(new_df['TransactionStartTime'])
  new_df['Hour'] = new_df['TransactionStartTime'].dt.hour
  
  
  #drop calculation columns
  new_df = new_df.drop(['value_string', 'value_last3','value_last4','value_last5','value_last6', 'TransactionStartTime'], axis=1)
  
  # Dummy variables added
  new_df = pd.get_dummies(new_df, columns = ['ProviderId'], prefix ='ProvID', drop_first = True)
  new_df = pd.get_dummies(new_df, columns = ['ProductId'], prefix ='ProdID', drop_first = True)
  new_df = pd.get_dummies(new_df, columns = ['ProductCategory'], prefix ='ProdCat', drop_first = True)
  new_df = pd.get_dummies(new_df, columns = ['ChannelId'], prefix ='ChanID', drop_first = True)
  new_df = pd.get_dummies(new_df, columns = ['PricingStrategy'], prefix ='PriceStrat', drop_first = True)
  new_df = pd.get_dummies(new_df, columns = ['Last_3'], prefix ='L3', drop_first = True)
  new_df = pd.get_dummies(new_df, columns = ['Last_4'], prefix ='L4', drop_first = True)
  new_df = pd.get_dummies(new_df, columns = ['Last_5'], prefix ='L5', drop_first = True)
  new_df = pd.get_dummies(new_df, columns = ['Last_6'], prefix ='L6', drop_first = True)
  new_df = pd.get_dummies(new_df, columns = ['Hour'], prefix ='Hour', drop_first = True)

  return new_df

In [0]:
combo = pd.concat([train,test], axis=0)

In [0]:
combo = manipulate_data(combo)

In [26]:
combo.head()

,Set,FraudResult,Value,ProvID_ProviderId_2,ProvID_ProviderId_3,ProvID_ProviderId_4,ProvID_ProviderId_5,ProvID_ProviderId_6,ProdID_ProductId_10,ProdID_ProductId_11,ProdID_ProductId_12,ProdID_ProductId_13,ProdID_ProductId_14,ProdID_ProductId_15,ProdID_ProductId_16,ProdID_ProductId_17,ProdID_ProductId_18,ProdID_ProductId_19,ProdID_ProductId_2,ProdID_ProductId_20,ProdID_ProductId_21,ProdID_ProductId_22,ProdID_ProductId_23,ProdID_ProductId_24,ProdID_ProductId_25,ProdID_ProductId_26,ProdID_ProductId_27,ProdID_ProductId_3,ProdID_ProductId_4,ProdID_ProductId_5,ProdID_ProductId_6,ProdID_ProductId_7,ProdID_ProductId_8,ProdID_ProductId_9,ProdCat_data_bundles,ProdCat_financial_services,ProdCat_movies,ProdCat_other,ProdCat_retail,ProdCat_ticket,ProdCat_transport,ProdCat_tv,ProdCat_utility_bill,ChanID_ChannelId_2,ChanID_ChannelId_3,ChanID_ChannelId_4,ChanID_ChannelId_5,PriceStrat_1,PriceStrat_2,PriceStrat_4,L3_Zeroes,L4_Zeroes,L5_Zeroes,L6_Zeroes,Hour_1,Hour_2,Hour_3,Hour_4,Hour_5,Hour_6,Hour_7,Hour_8,Hour_9,Hour_10,Hour_11,Hour_12,Hour_13,Hour_14,Hour_15,Hour_16,Hour_17,Hour_18,Hour_19,Hour_20,Hour_21,Hour_22,Hour_23
TransactionId,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
TransactionId_76871,train,0.0,1000,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
TransactionId_73770,train,0.0,20,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
TransactionId_26203,train,0.0,500,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
TransactionId_380,train,0.0,21800,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
TransactionId_28195,train,0.0,644,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [27]:
combo.iloc[np.r_[0:2, -2:0]]

,Set,FraudResult,Value,ProvID_ProviderId_2,ProvID_ProviderId_3,ProvID_ProviderId_4,ProvID_ProviderId_5,ProvID_ProviderId_6,ProdID_ProductId_10,ProdID_ProductId_11,ProdID_ProductId_12,ProdID_ProductId_13,ProdID_ProductId_14,ProdID_ProductId_15,ProdID_ProductId_16,ProdID_ProductId_17,ProdID_ProductId_18,ProdID_ProductId_19,ProdID_ProductId_2,ProdID_ProductId_20,ProdID_ProductId_21,ProdID_ProductId_22,ProdID_ProductId_23,ProdID_ProductId_24,ProdID_ProductId_25,ProdID_ProductId_26,ProdID_ProductId_27,ProdID_ProductId_3,ProdID_ProductId_4,ProdID_ProductId_5,ProdID_ProductId_6,ProdID_ProductId_7,ProdID_ProductId_8,ProdID_ProductId_9,ProdCat_data_bundles,ProdCat_financial_services,ProdCat_movies,ProdCat_other,ProdCat_retail,ProdCat_ticket,ProdCat_transport,ProdCat_tv,ProdCat_utility_bill,ChanID_ChannelId_2,ChanID_ChannelId_3,ChanID_ChannelId_4,ChanID_ChannelId_5,PriceStrat_1,PriceStrat_2,PriceStrat_4,L3_Zeroes,L4_Zeroes,L5_Zeroes,L6_Zeroes,Hour_1,Hour_2,Hour_3,Hour_4,Hour_5,Hour_6,Hour_7,Hour_8,Hour_9,Hour_10,Hour_11,Hour_12,Hour_13,Hour_14,Hour_15,Hour_16,Hour_17,Hour_18,Hour_19,Hour_20,Hour_21,Hour_22,Hour_23
TransactionId,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
TransactionId_76871,train,0.0,1000,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
TransactionId_73770,train,0.0,20,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
TransactionId_128439,test,NaN,5000,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
TransactionId_45927,test,NaN,3500,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1


In [0]:
X = combo[combo['Set']=='train'].drop(['FraudResult','Set'], axis=1)
y = combo[combo['Set']=='train']['FraudResult']
X_train, X_test, y_train, y_test = train_test_split(X, 
                                                    y,
                                                    test_size =0.1,
                                                   random_state = 42)

In [29]:
X.head()

,Value,ProvID_ProviderId_2,ProvID_ProviderId_3,ProvID_ProviderId_4,ProvID_ProviderId_5,ProvID_ProviderId_6,ProdID_ProductId_10,ProdID_ProductId_11,ProdID_ProductId_12,ProdID_ProductId_13,ProdID_ProductId_14,ProdID_ProductId_15,ProdID_ProductId_16,ProdID_ProductId_17,ProdID_ProductId_18,ProdID_ProductId_19,ProdID_ProductId_2,ProdID_ProductId_20,ProdID_ProductId_21,ProdID_ProductId_22,ProdID_ProductId_23,ProdID_ProductId_24,ProdID_ProductId_25,ProdID_ProductId_26,ProdID_ProductId_27,ProdID_ProductId_3,ProdID_ProductId_4,ProdID_ProductId_5,ProdID_ProductId_6,ProdID_ProductId_7,ProdID_ProductId_8,ProdID_ProductId_9,ProdCat_data_bundles,ProdCat_financial_services,ProdCat_movies,ProdCat_other,ProdCat_retail,ProdCat_ticket,ProdCat_transport,ProdCat_tv,ProdCat_utility_bill,ChanID_ChannelId_2,ChanID_ChannelId_3,ChanID_ChannelId_4,ChanID_ChannelId_5,PriceStrat_1,PriceStrat_2,PriceStrat_4,L3_Zeroes,L4_Zeroes,L5_Zeroes,L6_Zeroes,Hour_1,Hour_2,Hour_3,Hour_4,Hour_5,Hour_6,Hour_7,Hour_8,Hour_9,Hour_10,Hour_11,Hour_12,Hour_13,Hour_14,Hour_15,Hour_16,Hour_17,Hour_18,Hour_19,Hour_20,Hour_21,Hour_22,Hour_23
TransactionId,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
TransactionId_76871,1000,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
TransactionId_73770,20,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
TransactionId_26203,500,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
TransactionId_380,21800,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
TransactionId_28195,644,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [30]:
y.head()

TransactionId
TransactionId_76871    0.0
TransactionId_73770    0.0
TransactionId_26203    0.0
TransactionId_380      0.0
TransactionId_28195    0.0
Name: FraudResult, dtype: float64

In [0]:
def scoring_function_log_loss(y_test, y_pred_test):
    return log_loss(y_test, y_pred_test)

In [0]:
score_log_loss = make_scorer(scoring_function_log_loss, greater_is_better = False)

In [0]:
def tune_RF_model(X_train, y_train): 
    N_list = [1,10,100,250]
    random_state_list = [25,36,49,101,2504]

    score = make_scorer(scoring_function_log_loss, greater_is_better = False)
    
    rf = RandomForestClassifier()
    
    parameters = {'n_estimators':N_list,
                  'random_state': random_state_list}
    
    tune = GridSearchCV(rf, parameters, scoring = score)
    tune.fit(X_train,y_train)
    
    return tune

In [0]:
best_fraud_model = tune_RF_model(X_train, y_train)

In [63]:
best_fraud_model.best_params_

{'n_estimators': 10, 'random_state': 101}

In [0]:
fraud_model = RandomForestClassifier(n_estimators = best_fraud_model.best_params_['n_estimators'], random_state = best_fraud_model.best_params_['random_state'])

In [65]:
fraud_log_loss = cross_val_score(fraud_model, X, y, scoring=score_log_loss,cv=10)
print('Log Loss %2f' %(-1 * fraud_log_loss.mean()))

#fraud_acc = cross_val_score(fraud_model, X, y, scoring='accuracy',cv=10)
#print('Accuracy %2f' %(fraud_acc.mean()))

Log Loss 0.033217


In [0]:
predict_test = combo[combo['Set']=='test'].drop(['FraudResult','Set'], axis=1)

In [67]:
final_fraud_model = fraud_model
final_fraud_model.fit(X, y)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=None, max_features='auto', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=10,
                       n_jobs=None, oob_score=False, random_state=101,
                       verbose=0, warm_start=False)

In [0]:
final_fraud_predictions = final_fraud_model.predict(predict_test)

In [69]:
test_ID = predict_test.index.values
test_ID

array(['TransactionId_50600', 'TransactionId_95109',
       'TransactionId_47357', ..., 'TransactionId_102920',
       'TransactionId_128439', 'TransactionId_45927'], dtype=object)

In [0]:
my_submission = pd.DataFrame({'TransactionId': test_ID, 'FraudResult': final_fraud_predictions}) 

In [0]:
my_submission.set_index('TransactionId',inplace=True)

In [72]:
my_submission[my_submission['FraudResult']==1].count()

FraudResult    73
dtype: int64

In [0]:
from google.colab import files
my_submission.to_csv('rf_tune_sub_v2.csv') 
files.download('rf_tune_sub_v2.csv')

## Honour Code
I, Ernest John Lanz, confirm - by submitting my - that the solutions in this notebook are a result of my own work and that I abide by the EDSA honour code (https://drive.google.com/file/d/1QDCjGZJ8-FmJE3bZdIQNwnJyQKPhHZBn/view?usp=sharing).

Non-compliance with the honour code constitutes a material breach of contract.